In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
from pandas_profiling import ProfileReport
import json
import numpy as np
import seaborn as sns
sns.set(font="cmr10")
from seaborn import heatmap
from pprint import pprint as p
import os
import sys
import subprocess
from pdf2image import convert_from_path
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
rcParams.update({'font.family': 'cmr10'})
from __future__ import unicode_literals

# Load data
fn = 'tweets_01-08-2021.json'

with open(fn, encoding='utf-8') as f:
    df = pd.read_json(f)

# Drop retweets
df = df.loc[df.isRetweet == 'f']
df.drop(['isRetweet'], axis=1, inplace=True)

# Drop before Trump ran for President.
is_president_or_running_for_president = df['date'] < '2015-06-16'
df = df.loc[np.invert(is_president_or_running_for_president)]

# Drop noise due to links, utterannces, etc.
def map_fn(text):
    
    len_text = len(text.strip().split(" "))
    
    if len_text > 4:
        return False
    else:
        return True
    
noise_idx = df.text.map(map_fn)
noise = df.loc[noise_idx]
print(noise_idx.sum())
print(np.invert(noise_idx).sum())

df = df.loc[np.invert(noise_idx)].reset_index(drop=True)
df.sort_values(by=['retweets'], ascending=False, inplace=True)
p(df.info())
p(df.head())

2676
21888
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21888 entries, 4590 to 742
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         21888 non-null  int64         
 1   text       21888 non-null  object        
 2   isDeleted  21888 non-null  object        
 3   device     21888 non-null  object        
 4   favorites  21888 non-null  int64         
 5   retweets   21888 non-null  int64         
 6   date       21888 non-null  datetime64[ns]
 7   isFlagged  21888 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 1.5+ MB
None
                        id                                               text  \
4590   1311892190680014800  Tonight, @FLOTUS and I tested positive for COV...   
9074    795954831718498300                 TODAY WE MAKE AMERICA GREAT AGAIN!   
19659  1157345692517634000  A$AP Rocky released from prison and on his way...   
12975   9295

In [2]:
# # Print tweets randomized for viewing    
# df['merged_id_text'] = df['id'].astype(str) + ': ' + df['date'].astype(str) + ': '+ df['text'] + ': retweets= ' + df['retweets'].astype(str)
# df.merged_id_text.reset_index(drop=True).to_json('high_rt_tweets.json')
# df.info()

In [3]:
# Initialize new columns to false
features = [
    'isLie',
    'isMisspeaking',
    'isCantBeThatDumb',
    'isOpposite',
    'isInsinuating',
    'isPreemptive',
    'isElection',
    'isIKnowYouAreButWhatAmI',
    'isRacist',
    'isHitler',
    'isRussia',
    'isUkraine',
    'isChina',
    'isIran',
    'isNukes',
    'isExecutivePrivilege',
    'isSmear',
    'isSexist',
    'isCelebrity',
    'isPentagon',
    'isNickname',
    'isXenophobic',
    'isMAGA',
    'isReligious',
    'isPandemic',
    'isAllCaps',
    'isRINO',
    'isFirstImpeachment',
    'isSecondImpeachment',
    'isSoTrue',
    'isInTwoWeeks',
    'isWitchHunt',
    'isAntifa',
    'isBlackLivesMatter',
    'isLove',
    'isLügenpresse',
    'isDolchstoßlegende',
    'isBullying',
    'isTongueInCheek',
    'isThreatening',
    'isCommunism',
    'isCapitalRaid',
    'isWarPowersAct1973',
    'isWarCrime',
    'isHitlerBingo',
    'isIronic',
    'isKobraKai',
    'isWilliamWallace'
]

# Initialize new columns to false
df[features] = 'f'

feature_list = ['maga', 'make america great again']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature)
    is_magazine = df['text'].str.lower().str.contains('magazine')
    df.loc[is_maga, 'isMAGA'] = 't'
    df.loc[is_magazine, 'isMAGA'] = 'f'
print('isMAGA: ' + str(df['isMAGA'].str.contains('t').sum()))

# Populate witch hunt column
feature_list = ['witch hunt']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature)
    df.loc[is_maga, "isWitchHunt"] = 't'
print('isWitchHunt: ' + str(df['isWitchHunt'].str.contains('t').sum()))

# Populate in two weeks column
feature_list = ['two weeks', 'next week']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature)
    df.loc[is_maga, 'isInTwoWeeks'] = 't'
print('isInTwoWeeks: ' + str(df['isInTwoWeeks'].str.contains('t').sum()))

# Populate is so true column
feature_list = ['true.', 'so true']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature)
    df.loc[is_maga, 'isSoTrue'] = 't'
print('isSoTrue: ' + str(df['isSoTrue'].str.contains('t').sum()))

# Populate all caps column
for feature in feature_list:
    is_maga = df['text'].str.isupper()
    df.loc[is_maga, 'isAllCaps'] = 't'
print('isAllCaps: ' + str(df['isAllCaps'].str.contains('t').sum()))

# Populate is Russia column
feature_list = ['russia', 'russian', 'putin']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature)
    df.loc[is_maga, 'isRussia'] = 't'
print('isRussia: ' + str(df['isRussia'].str.contains('t').sum()))

# Populate is Ukraine column
feature_list = ['ukraine', 'ukrainian']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature) 
    df.loc[is_maga, 'isUkraine'] = 't'
print('isUkraine: ' + str(df['isUkraine'].str.contains('t').sum()))

# Populate is china column
feature_list = ['china', 'chinese']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature) 
    df.loc[is_maga, 'isChina'] = 't'
print('isChina: ' + str(df['isChina'].str.contains('t').sum()))

# Populate is iran column
feature_list = ['iran', 'iranian']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature) 
    df.loc[is_maga, 'isIran'] = 't'
print('isIran: ' + str(df['isIran'].str.contains('t').sum()))

# Populate is antifa column
feature_list = ['antifa']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature) 
    df.loc[is_maga, 'isAntifa'] = 't'
print('isAntifa: ' + str(df['isAntifa'].str.contains('t').sum()))

# Populate is love column
feature_list = ['love', 'heartwarming']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature) 
    df.loc[is_maga, 'isLove'] = 't'
print('isLove: ' + str(df['isLove'].str.contains('t').sum()))

# Populate is love column
feature_list = ['fake news', 'lamestream media', 'mainstream media']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature) 
    df.loc[is_maga, 'isLügenpresse'] = 't'
print('isLügenpresse (fake news, lametream media): ' + str(df['isLügenpresse'].str.contains('t').sum()))

# Populate is communism column
feature_list = ['communist', 'communism', 'marxist']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature) 
    df.loc[is_maga, 'isCommunism'] = 't'
print('isCommunism: ' + str(df['isCommunism'].str.contains('t').sum()))

# Populate is capital raid column
feature_list = ['january 6th', 'jan 6th']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature) 
    df.loc[is_maga, 'isCapitalRaid'] = 't'
print('isCapitalRaid: ' + str(df['isCapitalRaid'].str.contains('t').sum()))

# Populate is nukes column
feature_list = ['nuclear', 'nuke']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature) 
    df.loc[is_maga, 'isNukes'] = 't'
print('isNukes: ' + str(df['isNukes'].str.contains('t').sum()))

# Populate is religious column
feature_list = ['religion', 'god', 'prayer', 'pray']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature) 
    df.loc[is_maga, 'isReligious'] = 't'
print('isReligious: ' + str(df['isReligious'].str.contains('t').sum()))

# Populate is executive privilege column
feature_list = ['rino', 'republican in name only']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature) 
    df.loc[is_maga, 'isRINO'] = 't'
print('isRINO: ' + str(df['isRINO'].str.contains('t').sum()))


isMAGA: 909
isWitchHunt: 347
isInTwoWeeks: 61
isSoTrue: 312
isAllCaps: 117
isRussia: 512
isUkraine: 152
isChina: 571
isIran: 217
isAntifa: 28
isLove: 733
isLügenpresse (fake news, lametream media): 947
isCommunism: 16
isCapitalRaid: 8
isNukes: 104
isReligious: 302
isRINO: 53


In [4]:
# Load labels from more_features.json

with open('more_features.json', encoding='utf-8') as f:
    feature_dict = json.load(f)
                                  
for k, v in feature_dict.items():
    idx = df.loc[df['id'] == int(k)].index
    
    for feature in v:
        if type(feature) == dict:
            fname, soft_val = list(feature.items())[0]
            df.loc[idx, fname] = float(soft_val)
        else:
            df.loc[idx, feature] = 't'

label_df = df.loc[df['id'].isin(feature_dict.keys())].copy()

print('isLie: ' + str(df['isLie'].str.contains('t').sum()))
print('isSmear: ' + str(df['isSmear'].str.contains('t').sum()))
print('isDolchstoßlegende (The Big Lie): ' + str((df['isDolchstoßlegende'].str.contains('t')).sum()))

isLie: 190
isSmear: 122
isDolchstoßlegende (The Big Lie): 153


In [5]:
# Retain text, id, date, device
text_df = label_df.loc[:, ['id', 'text', 'date', 'device']]

# Drop columns not represented in the data, except twitter columns.
twitter_columns = [
    'isRetweet', 
    'isDeleted', 
    'favorites', 
    'retweets', 
    'isFlagged'
]

feature_list = []
for k, v in feature_dict.items():
    for feature in v:
        if isinstance(feature, dict):
            feature_list.append(list(feature.items())[0][0])
        else:
            feature_list.append(feature)

valid_features = list(set(feature_list).intersection(set(features)) | set(twitter_columns))

# Drop invalid columns
drop_columns = set(label_df.columns).difference(set(valid_features))
label_df.drop(list(drop_columns), axis=1, inplace=True)
print(label_df.columns.values)

['isDeleted' 'favorites' 'retweets' 'isFlagged' 'isLie' 'isOpposite'
 'isPreemptive' 'isElection' 'isIKnowYouAreButWhatAmI' 'isRacist'
 'isHitler' 'isRussia' 'isUkraine' 'isIran' 'isExecutivePrivilege'
 'isSmear' 'isCelebrity' 'isPentagon' 'isNickname' 'isXenophobic'
 'isPandemic' 'isAllCaps' 'isRINO' 'isFirstImpeachment' 'isAntifa'
 'isBlackLivesMatter' 'isLügenpresse' 'isDolchstoßlegende' 'isBullying'
 'isTongueInCheek' 'isThreatening' 'isWarPowersAct1973']


In [6]:
# Convert T/F to binary
def map_fn(tf_str):
    if tf_str == 't':
        return 1
    else:
        if tf_str == 'f':
            return 0
        else:
            return tf_str
    
for col in label_df:
    label_df.loc[:, col] = label_df[col].map(map_fn)

# Enforce that isDolchstoßlegende implies election lie
label_df.loc[label_df.isDolchstoßlegende == 1, ['isElection', 'isLie']] = 1

# Enforce that isLie > 0 implies isFalse
label_df.loc[label_df.isLie > 0, ['isFalse']] = 1

# Give Hitler points for triggers
isHitler = np.zeros((len(label_df), ))
if 'isAntifa' in label_df.columns.values:
    isHitler += label_df.isAntifa
if 'isLügenpresse' in label_df.columns.values:
    isHitler += label_df.isLügenpresse
if 'isDolchstoßlegende' in label_df.columns.values:
    isHitler += label_df.isDolchstoßlegende
if 'isBlackLivesMatter' in label_df.columns.values:
    isHitler += label_df.isBlackLivesMatter
label_df.loc[:, "isHitler"] = isHitler

# TODO: Define isHitlerBingo

label_df.head()

,isDeleted,favorites,retweets,isFlagged,isLie,isOpposite,isPreemptive,isElection,isIKnowYouAreButWhatAmI,isRacist,...,isFirstImpeachment,isAntifa,isBlackLivesMatter,isLügenpresse,isDolchstoßlegende,isBullying,isTongueInCheek,isThreatening,isWarPowersAct1973,isFalse
4590,0,1869706,408866,0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
9074,0,498035,281289,0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
19659,0,821423,226235,0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
12975,0,534338,217199,0,0.0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,NaN
1123,0,1188311,190721,1,1.0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1.0


In [7]:
profile = ProfileReport(label_df, title="Labeled Trump Tweets")
profile.to_file("label_profile_report.html")
profile.to_notebook_iframe()

In [ ]:
# LaTeX builder for correlation matrix/graph. 

# Compute stats and flag correlations @ p < .05
NUM_SAMPLES = 10000
SIGNIFICANCE = .95

samples = np.zeros((NUM_SAMPLES, len(label_df.columns.values)**2))
for i in range(NUM_SAMPLES):                   
    samples[i, :] = label_df.sample(n=len(label_df), replace=True).corr(min_periods=0).fillna(0).values.flatten()

# Compute significance of corr matrix
positive_sig = np.percentile(samples, 100*(1 - SIGNIFICANCE), axis=0) > 0
negative_sig = np.percentile(samples, 100*(SIGNIFICANCE), axis=0) < 0
corr_matrix_significance = np.reshape(np.logical_or(positive_sig, negative_sig), (len(label_df.columns.values), len(label_df.columns.values)))

plt.rc('text', usetex=True)
corr_heatmap = heatmap(
    label_df.corr().values * corr_matrix_significance.astype(float),
    vmin=-1.0,
    vmax=1.0, 
    center=0.0,
    square=True,
    cmap = 'RdBu',
    xticklabels=label_df.columns.values,
    yticklabels=label_df.columns.values
)
plt.rc('axes', unicode_minus=False)
plt.savefig('tex/corr_heatmap.pdf')
plt.gcf().set_visible(False)

# Graph view nodes
graph = '\\begin{tikzpicture}\n'
layout_rad = np.arange(0.0, 2*np.pi, 2*np.pi/len(label_df.columns.values))
xshift = np.cos(layout_rad)*5.5
yshift = np.sin(layout_rad)*5.5
for x, y, feature in zip(xshift, yshift, features):
    graph += '\\node[const] ({feature}) [xshift={x:.15f}cm, yshift={y:.15f}cm, minimum size=1.5cm] {{{feature}}}; '.format(x=x, y=y, feature=feature)

# Sample count for nodes
sample_count = (label_df.values > 0).sum(axis=0)
xshift = np.cos(layout_rad)*6.5
yshift = np.sin(layout_rad)*6.5
for x, y, count in zip(xshift, yshift, sample_count):
    graph += '\\node[const] ({count}) [xshift={x:.15f}cm, yshift={y:.15f}cm, minimum size=1.5cm] {{{count}}}; '.format(x=x, y=y, count=str(count))

corr_matrix = label_df.corr().values
for i in range(corr_matrix.shape[0]):
    for j in range(i):
        if corr_matrix_significance[i, j]:
            corr = corr_matrix[i, j]
            
            if corr < 0:
                color = 'red!{corr}!white'.format(corr=corr*-100)
            else:
                color = 'blue!{corr}!white'.format(corr=corr*100)

            graph += '\\edge [-, color={color}, line width={thickness:.15f}pt] {{{feature1}}} {{{feature2}}}; '.format(color=color, feature1=label_df.columns.values[i], feature2=label_df.columns.values[j], thickness=np.abs(corr)*2)


with open ('tex/correlation_graph.tex', 'w', encoding='utf-8') as f:
    f.write(graph + '\n\\end{tikzpicture}')

# Hitler subgraph
graph = '\\begin{tikzpicture}'
hitler_features = ['isHitler', 'isLügenpresse', 'isDolchstoßlegende', 'isLie', 'isElection', 'isFlagged']
hitler_features_idx = [label_df.columns.values.tolist().index(x) for x in hitler_features]
hitler_corr_significance = corr_matrix_significance[:, hitler_features_idx][hitler_features_idx, :]
hitler_corr = corr_matrix[:, hitler_features_idx][hitler_features_idx, :]

graph += '\\node[const] (isHitler) [minimum size=1.5cm] {isHitler};\n'
graph += '\\node[const] (isDolchstoßlegende) [right=of isHitler, minimum size=1.5cm] {isDolchstoßlegende};\n'
graph += '\\node[const] (isLügenpresse) [left=of isHitler, minimum size=1.5cm] {isLügenpresse};\n'
graph += '\\node[const] (isLie) [above=of isHitler, minimum size=1.5cm, xshift=-2cm] {isLie};\n'
graph += '\\node[const] (isElection) [below=of isHitler, minimum size=1.5cm] {isElection};\n'
graph += '\\node[const] (isFlagged) [above=of isHitler, minimum size=1.5cm, xshift=2cm] {isFlagged};\n'

for i in range(hitler_corr.shape[0]):
    for j in range(i):
        if hitler_corr_significance[i, j]:
            corr = hitler_corr[i, j]
            
            if corr < 0:
                color = 'red!{corr}!white'.format(corr=corr*-100)
            else:
                color = 'blue!{corr}!white'.format(corr=corr*100)

            graph += '\\edge [-, color={color}, line width={thickness:.15f}pt] {{{feature1}}} {{{feature2}}}; '.format(color=color, feature1=hitler_features[i], feature2=hitler_features[j], thickness=np.abs(corr)*2)
        
with open ('tex/hitler_correlation_graph.tex', 'w', encoding='utf-8') as f:
    f.write(graph + '\n\\end{tikzpicture}')

# Compile latex
os.system('cd tex && xelatex main.tex')

images = convert_from_path('tex/main.pdf')
for i in range(len(images)):
    width, height = images[i].size
    display(images[i].resize((2*width, 2*height)))
    images[i].save('tex/main_' + str(i) + '.jpg')
